In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# 날씨데이터 (다운로드)

In [48]:
weather = pd.read_csv('../data/날씨데이터(기상자료개방포털).csv', encoding='cp949')

In [49]:
weather = weather[['일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)']]
weather.columns = ['date', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)']

In [50]:
import datetime
weather['date'] = pd.to_datetime(weather['date'])

In [51]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5856 entries, 0 to 5855
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     5856 non-null   datetime64[ns]
 1   기온(°C)   5855 non-null   float64       
 2   강수량(mm)  865 non-null    float64       
 3   풍속(m/s)  5856 non-null   float64       
 4   습도(%)    5856 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 228.9 KB


In [56]:
# weather['시간'] = ''
for i in range(len(weather)):
#     weather['시간'][i] = weather['date'][i].hour
    weather['date'][i] = weather['date'][i].date()

C:\Users\jejec\anaconda3\envs\mulcam\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [57]:
# 전체 데이터프레임도 시간 앞 숫자 따서 변경할 것!
weather

,date,기온(°C),강수량(mm),풍속(m/s),습도(%),시간
0,2020-01-01,-6.5,0.0,0.0,38,0
1,2020-01-01,-5.9,NaN,1.7,40,1
2,2020-01-01,-5.7,NaN,0.1,42,2
3,2020-01-01,-5.6,0.0,0.0,46,3
4,2020-01-01,-5.4,NaN,0.0,50,4
...,...,...,...,...,...,...
5851,2020-08-31,28.0,NaN,3.5,73,19
5852,2020-08-31,27.6,NaN,2.5,73,20
5853,2020-08-31,27.1,NaN,2.4,74,21
5854,2020-08-31,26.8,NaN,2.3,76,22


In [58]:
weather = weather[['date', '시간', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)']]
weather

,date,시간,기온(°C),강수량(mm),풍속(m/s),습도(%)
0,2020-01-01,0,-6.5,0.0,0.0,38
1,2020-01-01,1,-5.9,NaN,1.7,40
2,2020-01-01,2,-5.7,NaN,0.1,42
3,2020-01-01,3,-5.6,0.0,0.0,46
4,2020-01-01,4,-5.4,NaN,0.0,50
...,...,...,...,...,...,...
5851,2020-08-31,19,28.0,NaN,3.5,73
5852,2020-08-31,20,27.6,NaN,2.5,73
5853,2020-08-31,21,27.1,NaN,2.4,74
5854,2020-08-31,22,26.8,NaN,2.3,76


In [64]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5856 entries, 0 to 5855
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     5856 non-null   datetime64[ns]
 1   시간       5856 non-null   object        
 2   기온(°C)   5855 non-null   float64       
 3   강수량(mm)  865 non-null    float64       
 4   풍속(m/s)  5856 non-null   float64       
 5   습도(%)    5856 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 274.6+ KB


In [65]:
weather['강수량(mm)'] = weather['강수량(mm)'].fillna(0)
weather[weather['기온(°C)'].isnull()]

,date,시간,기온(°C),강수량(mm),풍속(m/s),습도(%)
5725,2020-08-26,13,NaN,0.0,4.3,60


In [69]:
weather[5720:5730]

,date,시간,기온(°C),강수량(mm),풍속(m/s),습도(%)
5720,2020-08-26,8,28.6,0.0,2.7,79
5721,2020-08-26,9,29.7,0.0,3.2,74
5722,2020-08-26,10,31.4,0.0,3.5,67
5723,2020-08-26,11,32.6,0.0,3.8,62
5724,2020-08-26,12,33.3,0.0,3.0,60
5725,2020-08-26,13,NaN,0.0,4.3,60
5726,2020-08-26,14,34.1,0.0,4.0,58
5727,2020-08-26,15,33.5,0.0,3.1,57
5728,2020-08-26,16,34.4,0.0,4.6,54
5729,2020-08-26,17,33.4,0.0,4.5,54


In [70]:
weather['기온(°C)'] = weather['기온(°C)'].fillna(method='ffill')

In [74]:
weather.isna().sum()

date       0
시간         0
기온(°C)     0
강수량(mm)    0
풍속(m/s)    0
습도(%)      0
dtype: int64

In [75]:
# weather.to_pickle('날씨(기상자료개방포털).pkl')